In [107]:
import pandas as pd
import io

# Load the file using the dynamically obtained filename
df_train = pd.read_csv("op1.csv")
df_test = pd.read_csv("op2.csv")


# Checking Nulls to see need for imputation
print(df_train.isnull().sum())

Id        0
q1        0
q2        0
q3        0
q4        0
q5        0
q6        0
q7        0
q8        0
q9        0
q10       0
q11       0
q12       0
q13       0
q14       0
q15       0
q16       0
q17       0
q18       0
q19       0
q20       0
q21      10
q22      28
label     0
dtype: int64


In [111]:
df_train = df_train.drop(df_train.columns[[21, 22]], axis = 1) #removing column 21 & 22
df_train.head()

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q12,q13,q14,q15,q16,q17,q18,q19,q20,label
0,bfe3aa97-eb08-43d9-88a5-34ce93509321,6,6.000000,1,2,2,2,2,2,1,...,3,2,2,2,1,2,1,2,1,3
1,3b47ee28-e621-4497-949b-0fa0a59044f7,0,4.000000,1,2,2,2,0,1,1,...,1,2,2,2,0,1,1,2,2,3
2,e21808f4-2c7f-4cdf-90f5-daa7ccccc843,0,5.000000,2,2,2,2,0,4,2,...,2,2,1,2,2,2,2,2,2,4
3,59385c6a-5a11-4348-b0d1-c1f87b56cbd3,5,3.764706,0,1,1,1,4,2,1,...,1,0,1,3,2,3,3,3,1,2
4,6a769ce6-900b-4490-9d67-6a9e8190b78c,0,0.000000,1,2,2,2,2,2,1,...,2,1,1,2,2,1,1,2,1,3


In [113]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# Initialize LeaveOneOut cross-validator
loo = LeaveOneOut()

# Features and target
X = df_train.drop(['Id', 'label'], axis=1)
y = df_train['label']

# Lists to store results
accuracies = []

# Perform Leave One Out Cross Validation
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit your model here (example using a classifier)
    model_rf = RandomForestClassifier()
    model_rf.fit(X_train, y_train)
    
    # Predict on the test set
    rf_pred = model_rf.predict(X_test)
    
    # Evaluate the model (example using accuracy)
    accuracy = accuracy_score(y_test, rf_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f'Average Accuracy: {average_accuracy}')

print("Classification Report :")
print(classification_report(y_test, rf_pred))
print("Accuracy :")
print(accuracy_score(y_test, rf_pred))
print("Cohen Kappa Score :")
print(cohen_kappa_score(y_test, rf_pred))

Average Accuracy: 0.92
Classification Report :
              precision    recall  f1-score   support

           4       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

Accuracy :
1.0
Cohen Kappa Score :
nan


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:708: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


In [73]:
import pandas as pd

# Assuming df_test is your test dataset and 'model' is the trained model
# Remove the 'ID' column from the test data (but store it for later use)
ids = df_test['Id']  # Save the ID column before dropping it
df_test = df_test.drop(columns=['Id', 'q21', 'q22'])

# Make predictions on the test data
y_pred = model_rf.predict(df_test)

# Store the predictions in an array 'label'
label = y_pred

# Create a DataFrame to join the IDs and the predictions
result_df = pd.DataFrame({'Id': ids, 'label': label})

# Now, result_df contains the ID and its corresponding prediction
print(result_df)


                                        Id  label
0     a2c83973-8e81-45c9-bb84-37fa8a8b637c      1
1     78bca2d4-8824-45ed-80c9-72ef0e4389c6      4
2     69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36      3
3     d5d506d9-29bb-404d-9061-3db9ae92d711      3
4     d0796afc-055d-4ea8-9552-336ad0442de5      1
...                                    ...    ...
2945  1885d9ad-c858-4cc9-877b-491e912386a5      4
2946  396a3f34-6f62-4785-9707-15f22e1f23b9      3
2947  5ad78da8-4a84-4685-b2b3-7e49590915f7      3
2948  55188d8a-8895-4289-acf5-9606972dca8e      5
2949  8ea006cd-7233-4b18-9826-bd8206a28483      1

[2950 rows x 2 columns]


In [97]:
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier  # You can use any classifier
import pandas as pd

# Prepare your data
X = df_train.drop(['Id', 'label'], axis=1)
y = df_train['label']

# Initialize your model (example: Random Forest)
model = RandomForestClassifier(random_state=0)

# Define parameter grid for tuning
param_grid = {
    'n_estimators': [40, 45, 50, 55],  # Number of trees in the forest
    'max_depth': [4, 5, 6, 7, 10],        # Maximum depth of the trees
    'min_samples_split': [2, 3, 4, 5],  # Minimum samples required to split an internal node
}

# Initialize Leave-One-Out Cross Validation (LOO)
loo = LeaveOneOut()

# Set up the GridSearchCV with the LOO-CV strategy
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='accuracy', cv=loo, n_jobs=-1)

# Fit the grid search model
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Output the results
print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score}")


Best Parameters: {'max_depth': 4, 'min_samples_split': 2, 'n_estimators': 55}
Best Score: 0.96


In [99]:
# Assuming you already have the best model from grid_search.best_estimator_
best_model = grid_search.best_estimator_

# Assuming df_test is your test dataset and it has an 'Id' column and 'label' column (not used for prediction)

# Preprocess the test data the same way as training data, if any preprocessing was done
# For example, if scaling or encoding was applied, apply it here as well

# Make predictions on the test data using the best trained model
y_pred_test = best_model.predict(df_test)

# Store the predictions in a 'label' array
label_test = y_pred_test

# Create a DataFrame to join the IDs and the predictions
result_df_test = pd.DataFrame({'Id': ids, 'label': label_test})

# Now, result_df_test contains the ID and its corresponding prediction
print(result_df_test)

                                        Id  label
0     a2c83973-8e81-45c9-bb84-37fa8a8b637c      1
1     78bca2d4-8824-45ed-80c9-72ef0e4389c6      4
2     69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36      3
3     d5d506d9-29bb-404d-9061-3db9ae92d711      3
4     d0796afc-055d-4ea8-9552-336ad0442de5      1
...                                    ...    ...
2945  1885d9ad-c858-4cc9-877b-491e912386a5      4
2946  396a3f34-6f62-4785-9707-15f22e1f23b9      2
2947  5ad78da8-4a84-4685-b2b3-7e49590915f7      3
2948  55188d8a-8895-4289-acf5-9606972dca8e      5
2949  8ea006cd-7233-4b18-9826-bd8206a28483      1

[2950 rows x 2 columns]


In [75]:
result_df.to_csv("submission_1.csv", index=False)

In [89]:
result_df_test.to_csv("submission_2.csv", index=False)

In [101]:
result_df_test.to_csv("submission_3.csv", index=False)

In [119]:
import pandas as pd

# Load the first CSV (with 22 columns and ID)
df1 = pd.read_csv('op2.csv')  # Replace with your file name

# Load the second CSV (with ID and label)
df2 = pd.read_csv('submission_2.csv')  # Replace with your file name

# Merge the dataframes on 'ID', but avoid repeating the 'ID' column by specifying 'on' and 'how' as 'left'
merged_df = pd.merge(df1, df2[['Id', 'label']], on='Id', how='left')

# Display the merged DataFrame
print(merged_df.head())

                                     Id  q1        q2  q3  q4  q5  q6  q7  q8  \
0  a2c83973-8e81-45c9-bb84-37fa8a8b637c   3  3.683624   0   1   1   1   4   4   
1  78bca2d4-8824-45ed-80c9-72ef0e4389c6   1  6.000000   2   2   2   1   0   0   
2  69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36   3  2.000000   1   2   2   2   2   2   
3  d5d506d9-29bb-404d-9061-3db9ae92d711   4  3.683624   0   2   1   2   2   0   
4  d0796afc-055d-4ea8-9552-336ad0442de5   1  3.683624   0   1   1   0   1   4   

   q9  ...  q14  q15  q16  q17  q18  q19  q20  \
0   1  ...    3    1    1    0    0    0    0   
1   2  ...    1    2    2    2    2    2    2   
2   2  ...    2    2    2    1    1    2    2   
3   0  ...    2    0    0    2    3    0    2   
4   1  ...    3    0    1    1    1    0    0   

                                                 q21  \
0                                                NaN   
1  ['other surviving family units', 'One particul...   
2  ['termz ov grief processin', 'sense ov camarad.

In [121]:
merged_df = merged_df.drop(merged_df.columns[[21, 22]], axis = 1) #removing column 21 & 22
merged_df.head()

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q12,q13,q14,q15,q16,q17,q18,q19,q20,label
0,a2c83973-8e81-45c9-bb84-37fa8a8b637c,3,3.683624,0,1,1,1,4,4,1,...,2,0,3,1,1,0,0,0,0,1
1,78bca2d4-8824-45ed-80c9-72ef0e4389c6,1,6.000000,2,2,2,1,0,0,2,...,2,2,1,2,2,2,2,2,2,4
2,69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36,3,2.000000,1,2,2,2,2,2,2,...,2,2,2,2,2,1,1,2,2,3
3,d5d506d9-29bb-404d-9061-3db9ae92d711,4,3.683624,0,2,1,2,2,0,0,...,1,3,2,0,0,2,3,0,2,3
4,d0796afc-055d-4ea8-9552-336ad0442de5,1,3.683624,0,1,1,0,1,4,1,...,1,1,3,0,1,1,1,0,0,1


In [131]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score, confusion_matrix
from sklearn.metrics import roc_auc_score

# Initialize KFold cross-validator with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Features and target
X = merged_df.drop(['Id', 'label'], axis=1)
y = merged_df['label']

# Lists to store results
accuracies = []
roc_auc_scores = []

# Perform 10-Fold Cross Validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit your model here (RandomForestClassifier in this case)
    model_rf = RandomForestClassifier()
    model_rf.fit(X_train, y_train)
    
    # Predict on the test set
    rf_pred = model_rf.predict(X_test)
    
    # Evaluate the model (example using accuracy and ROC AUC)
    accuracy = accuracy_score(y_test, rf_pred)
    accuracies.append(accuracy)
    
    # Calculate ROC AUC score (if possible for multi-class classification)
    try:
        auc_score = roc_auc_score(y_test, model_rf.predict_proba(X_test), multi_class='ovr', average='macro')
        roc_auc_scores.append(auc_score)
    except ValueError:
        # In case ROC AUC is not applicable for a particular type of classification (e.g., binary or multi-class)
        roc_auc_scores.append(None)

# Calculate the average accuracy and ROC AUC score
average_accuracy = sum(accuracies) / len(accuracies)
average_auc_score = sum(filter(None, roc_auc_scores)) / len(roc_auc_scores) if roc_auc_scores else None

# Output the results
print(f'Average Accuracy: {average_accuracy}')
if average_auc_score is not None:
    print(f'Average ROC AUC Score: {average_auc_score}')

# Generate the classification report for the last fold
print("Classification Report:")
print(classification_report(y_test, rf_pred))

# Calculate Cohen Kappa Score for the last fold
print("Cohen Kappa Score:")
print(cohen_kappa_score(y_test, rf_pred))

# Confusion Matrix for the last fold
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_pred))

Average Accuracy: 0.963050847457627
Average ROC AUC Score: 0.9983378056492362
Classification Report:
              precision    recall  f1-score   support

           1       0.93      0.98      0.95        41
           2       0.91      0.91      0.91        47
           3       0.97      0.92      0.95        75
           4       0.92      0.96      0.94        48
           5       0.98      1.00      0.99        53
           6       1.00      0.97      0.98        31

    accuracy                           0.95       295
   macro avg       0.95      0.96      0.95       295
weighted avg       0.95      0.95      0.95       295

Cohen Kappa Score:
0.9422668306866473
Confusion Matrix:
[[40  0  1  0  0  0]
 [ 2 43  0  1  1  0]
 [ 1  2 69  3  0  0]
 [ 0  1  1 46  0  0]
 [ 0  0  0  0 53  0]
 [ 0  1  0  0  0 30]]


In [133]:
import pandas as pd

# Assuming df_test is your test dataset and 'model' is the trained model
# Remove the 'ID' column from the test data (but store it for later use)
ids = df_test['Id']  # Save the ID column before dropping it
df_test = df_test.drop(columns=['Id', 'q21', 'q22'])

# Make predictions on the test data
y_pred = model_rf.predict(df_test)

# Store the predictions in an array 'label'
label = y_pred

# Create a DataFrame to join the IDs and the predictions
result_df = pd.DataFrame({'Id': ids, 'label': label})

# Now, result_df contains the ID and its corresponding prediction
print(result_df)

result_df.to_csv("submission_4.csv", index=False)

                                        Id  label
0     a2c83973-8e81-45c9-bb84-37fa8a8b637c      1
1     78bca2d4-8824-45ed-80c9-72ef0e4389c6      4
2     69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36      3
3     d5d506d9-29bb-404d-9061-3db9ae92d711      3
4     d0796afc-055d-4ea8-9552-336ad0442de5      1
...                                    ...    ...
2945  1885d9ad-c858-4cc9-877b-491e912386a5      4
2946  396a3f34-6f62-4785-9707-15f22e1f23b9      3
2947  5ad78da8-4a84-4685-b2b3-7e49590915f7      3
2948  55188d8a-8895-4289-acf5-9606972dca8e      5
2949  8ea006cd-7233-4b18-9826-bd8206a28483      1

[2950 rows x 2 columns]


In [138]:
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier  # You can use any classifier
import pandas as pd

# Prepare your data
X = merged_df.drop(['Id', 'label'], axis=1)
y = merged_df['label']

# Initialize your model (example: Random Forest)
model = RandomForestClassifier(random_state=0)

# Define parameter grid for tuning
param_grid = {
    'n_estimators': [40, 45, 50, 55],  # Number of trees in the forest
    'max_depth': [4, 5, 6, 7, 10],        # Maximum depth of the trees
    'min_samples_split': [2, 3, 4, 5],  # Minimum samples required to split an internal node
}

# Initialize KFold cross-validation (10-fold)
kf = KFold(n_splits=10, shuffle=True, random_state=0)

# Set up the GridSearchCV with the 10-fold CV strategy
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='accuracy', cv=kf, n_jobs=-1)

# Fit the grid search model
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Output the results
print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score}")

Best Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 45}
Best Score: 0.9596610169491526


In [139]:
# Assuming you already have the best model from grid_search.best_estimator_
best_model = grid_search.best_estimator_

# Assuming df_test is your test dataset and it has an 'Id' column and 'label' column (not used for prediction)

# Preprocess the test data the same way as training data, if any preprocessing was done
# For example, if scaling or encoding was applied, apply it here as well

# Make predictions on the test data using the best trained model
y_pred_test = best_model.predict(df_test)

# Store the predictions in a 'label' array
label_test = y_pred_test

# Create a DataFrame to join the IDs and the predictions
result_df_test = pd.DataFrame({'Id': ids, 'label': label_test})

# Now, result_df_test contains the ID and its corresponding prediction
print(result_df_test)

result_df_test.to_csv("submission_5.csv", index=False)

                                        Id  label
0     a2c83973-8e81-45c9-bb84-37fa8a8b637c      1
1     78bca2d4-8824-45ed-80c9-72ef0e4389c6      4
2     69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36      3
3     d5d506d9-29bb-404d-9061-3db9ae92d711      3
4     d0796afc-055d-4ea8-9552-336ad0442de5      1
...                                    ...    ...
2945  1885d9ad-c858-4cc9-877b-491e912386a5      4
2946  396a3f34-6f62-4785-9707-15f22e1f23b9      3
2947  5ad78da8-4a84-4685-b2b3-7e49590915f7      3
2948  55188d8a-8895-4289-acf5-9606972dca8e      5
2949  8ea006cd-7233-4b18-9826-bd8206a28483      1

[2950 rows x 2 columns]
